In [ ]:
!pip install evaluate
!pip install rouge_score

In [1]:
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader
from collections import defaultdict
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
import torch
import torch.nn as nn
from transformers import pipeline

In [2]:
dataset = load_dataset("OpenHust/vietnamese-summarization", data_files="bio_medicine.csv")

Found cached dataset csv (/home/jupyter/.cache/huggingface/datasets/OpenHust___csv/OpenHust--vietnamese-summarization-0ae3a3f3a71406ca/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Document', 'Summary', 'Dataset'],
        num_rows: 10652
    })
})

In [90]:
small= dataset["train"]

train, test = small.train_test_split(train_size=0.8, seed=0).values()
train, dev = train.train_test_split(test_size=0.125, seed=0).values()

In [91]:
train

Dataset({
    features: ['Document', 'Summary', 'Dataset'],
    num_rows: 7455
})

from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base").to("cuda")

from transformers import XGLMTokenizer, XGLMForCausalLM

tokenizer = XGLMTokenizer.from_pretrained("facebook/xglm-564M")
model = XGLMForCausalLM.from_pretrained("facebook/xglm-564M")

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2TokenizerFast

tokenizer = GPT2TokenizerFast.from_pretrained('OpenHust/viet-gpt2', pad_token_id=50256 ,padding_side="right")
model = GPT2LMHeadModel.from_pretrained('OpenHust/viet-gpt2', pad_token_id=50256 , max_length=1024).to("cuda")


In [98]:
class bio_medicine(Dataset):
    
    def __init__(self, docs, sums):
        indices = [idx for idx, d in enumerate(docs)]
        self.docs = [docs[i] for i in indices]
        self.sums = [sums[i] for i in indices]
        self.tokenizer = GPT2Tokenizer.from_pretrained('NlpHUST/gpt2-vietnamese')
        self.tokenizer.add_special_tokens({"eos_token":"<s>"})
    
    def __len__(self):
        return len(self.docs)
    
    def __getitem__(self, idx):
        item = {}
        #encoder-decoder
        source = self.tokenizer(self.docs[idx])
        target = self.tokenizer(self.sums[idx])
        item["input_ids"] = source["input_ids"]
        item["attention_mask"] = source["attention_mask"]
        item["decoder_input_ids"] = target["input_ids"]
        item["decoder_attention_mask"] = target["attention_mask"]
        #end encoder-decoder
        length = len(item["attention_mask"][:500])
        ori = item["input_ids"]
        item["input_ids"] = [44282, 2589, 910, 781, 639, 1315] + item["input_ids"][:500]+[7720, 781, 5307, 2589, 1315] + item["decoder_input_ids"][:500]+[0]
        item["input_ids"] = item["input_ids"]# + [0] * (1023-len(item["input_ids"]))
        item["attention_mask"] = item["attention_mask"][:500] + [1]*11 + item["decoder_attention_mask"][:500]+[1]
        item["attention_mask"] = item["attention_mask"]# + [0] * (1023-len(item["attention_mask"]))
        item["labels"] = [-100]*6 + [-100]*len(ori[:500])+[-100]*5 + item["decoder_input_ids"][:500]+[0]
        if len(item["input_ids"]) != len(item["labels"]):
            print("Diff")
        return {"input_ids":item["input_ids"], "attention_mask":item["attention_mask"], "labels":item["labels"]}
    
    def get_data(self, batch):
        item = {}
        for k in batch[0]:
            item[k] = [batch[i][k] for i in range(len(batch))]
            
        return item
    
    def collate_fn(self, batch):
        batch = self.get_data(batch)
        try:
            emax = max([len(i) for i in batch["input_ids"]])
        except:
            import pdb
            pdb.set_trace()
        if False:
            batch["input_ids"] = torch.tensor([(i + [0]*(emax - len(i)))[:512] for i in batch["input_ids"]])
            batch["attention_mask"] = torch.tensor([(i + [0]*(emax - len(i)))[:512] for i in batch["attention_mask"]])

            dmax = max([len(i) for i in batch["decoder_input_ids"]])
            batch["decoder_input_ids"] = torch.tensor([(i + [0]*(emax - len(i)))[:512] for i in batch["decoder_input_ids"]])
            batch["decoder_attention_mask"] = torch.tensor([(i + [0]*(emax - len(i)))[:512] for i in batch["decoder_attention_mask"]])
            batch["labels"] =  batch["decoder_input_ids"].clone()
        else:
            other = {}
            other["input_ids"] = torch.tensor([([50256]*(emax - len(i))+i) for i in batch["input_ids"]])
            other["attention_mask"] = torch.tensor([([0]*(emax - len(i))+i) for i in batch["attention_mask"]])
            other["labels"] = torch.tensor([([-100]*(emax - len(i))+i) for i in batch["labels"]])
            batch = other
        if batch["input_ids"].size(0) == 0:
            import pdb
            pdb.set_trace()
        return batch

In [ ]:
train_dataset = bio_medicine(train["Document"], train["Summary"])
test_dataset = bio_medicine(test["Document"], test["Summary"])
dev_dataset = bio_medicine(dev["Document"], dev["Summary"])

In [100]:
len(test_dataset)

2131

In [101]:
for batch in train_dataset:
    break

In [102]:
for param in model.parameters():
    print(param)
    break

Parameter containing:
tensor([[ 0.4279,  0.1275,  0.1712,  ..., -0.1612, -0.1743, -0.1371],
        [ 0.0576,  0.0699,  0.0041,  ..., -0.1084, -0.1452,  0.0853],
        [ 0.1624,  0.1526,  0.1156,  ..., -0.0452, -0.1966,  0.0380],
        ...,
        [ 0.2943,  0.2226,  0.7038,  ...,  0.1234,  0.2001,  0.2527],
        [ 0.3256,  0.4152,  0.3054,  ...,  0.1602, -0.2325,  0.5641],
        [-0.4826,  0.2865, -0.5087,  ..., -0.2125, -0.5416,  0.1025]],
       device='cuda:0', requires_grad=True)


In [103]:
tokenizer.eos_token_id

0

In [104]:
import nltk
import numpy as np
import evaluate

metric = evaluate.load("rouge")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    #predictions = predictions[:, :-1]
    
    predictions[predictions == -100] = 0
    predictions[labels == -100] = 0
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    #labels = labels[labels !=-100]
    labels = np.where(labels != -100, labels, tokenizer.eos_token_id)
    
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    #import pdb
    #pdb.set_trace()
    return {k: round(v, 4) for k, v in result.items()}
import nltk
import numpy as np
import evaluate
from datasets import load_metric

metric = evaluate.load("rouge")

def compute_bleu(eval_preds):
    
    y_true = eval_preds.label_ids
    y_pred = eval_preds.predictions
    y_true = np.where(y_true != -100, y_true, tokenizer.pad_token_id)
    y_pred = np.where(y_pred != -100, y_true, tokenizer.pad_token_id)
    #import pdb
    #pdb.set_trace()
    metric = load_metric('bleu')
    #import pdb
    #pdb.set_trace()
    try:
        y_true = tokenizer.batch_decode(y_true, skip_special_tokens=True)
        y_pred = tokenizer.batch_decode(y_pred, skip_special_tokens=True)
        y_true = [[i.split()] for i in y_true]
        y_pred= [i.split() for i in y_pred]
        report = metric.compute(predictions=y_pred, references=y_true)
    except Exception as e:
        print(e)
        import pdb
        pdb.set_trace()
    bleu = report['bleu'] * 100
    print(bleu)
    return {"bleu":bleu}

In [105]:
args = Seq2SeqTrainingArguments(output_dir="kaggle/working",
                                evaluation_strategy="steps",
                                save_steps = 1917*5,
                                eval_steps = 200,
                               per_device_train_batch_size=4,
                               per_device_eval_batch_size=4,
                               learning_rate=1e-4,
                               weight_decay=1e-2,
                               #load_best_model_at_end =True,
                               predict_with_generate=True,
                                 generation_max_length=1024,
                               num_train_epochs=5,
                               fp16=True)

trainer = Seq2SeqTrainer(model=model, 
                        args=args,
                        data_collator=train_dataset.collate_fn,
                        train_dataset=train_dataset,
                        eval_dataset=dev_dataset,
                        tokenizer=tokenizer,
                        compute_metrics=compute_metrics)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using amp half precision backend


In [ ]:
trainer.train()

In [106]:
trainer.evaluate(test_dataset)

***** Running Evaluation *****
  Num examples = 2131
  Batch size = 4


{'eval_loss': 2.0273325443267822,
 'eval_rouge1': 0.5531,
 'eval_rouge2': 0.5509,
 'eval_rougeL': 0.5528,
 'eval_rougeLsum': 0.5534,
 'eval_gen_len': 1024.0,
 'eval_runtime': 3035.1453,
 'eval_samples_per_second': 0.702,
 'eval_steps_per_second': 0.176}

In [22]:
dev["Document"][6]

'Bạn không nên uống bất kì loại thuốc chống đông máu nào trong ít nhất vài ngày sau phẫu thuật. Nhờ bác sĩ tư vấn về việc này nếu đang sử dụng thuốc chống đông máu, vì nó làm tăng nguy cơ chảy máu hậu phẫu thuật. Tùy vào mỗi người thời gian cần ngừng uống thuốc chống đông máu sẽ khác nhau (thứ nhất là phụ thuộc vào lý do vì sao bạn phải dùng loại thuốc này). Nhờ bác sĩ hướng dẫn khi nào có thể uống thuốc trở lại bình thường. Nghỉ ngơi là một trong những yếu tố quan trọng nhất để phục hồi sau phẫu thuật cắt ống dẫn tinh. Bạn nên nghỉ phép vài ngày hoặc hạn chế làm các công việc thường nhật để vết thương lành nhanh hơn. Trừ khi công việc đòi hỏi phải hoạt động nhiều hoặc nâng nặng, bạn có thể đi làm lại khá nhanh, khoảng 2-3 ngày sau. Nếu phải làm việc nặng bạn nên hỏi bác sĩ khi nào an toàn để đi làm lại.  Cố gắng không làm việc quá nhiều trong 2-3 ngày đầu tiên sau phẫu thuật, và không ngại nhờ người khác giúp đỡ để có thời gian nghỉ ngơi phục hồi tốt hơn. Khuyến cáo hoạt động thể chất

In [23]:
len(dev["Document"][6].split())

634

In [24]:
len(dev["Summary"][6].split())

34

In [25]:
input_ids = tokenizer.encode("tóm tắt văn bản sau :"+ dev["Document"][6] + "văn bản tóm tắt :", return_tensors='pt').to("cuda")
max_length = 800

sample_outputs = model.generate(input_ids,pad_token_id=tokenizer.eos_token_id,
                                eos_token_id=0,
                                   do_sample=True,
                                   max_length=max_length,
                                   top_k=40,
                                   num_beams=5,
                                   early_stopping=True,
                                   no_repeat_ngram_size=2,
                                   num_return_sequences=3)

for i, sample_output in enumerate(sample_outputs):
    print(">> Generated text {}\n\n{}".format(i+1, tokenizer.decode(sample_output.tolist())))
    print('\n---')


>> Generated text 1

tóm tắt văn bản sau :Bạn không nên uống bất kì loại thuốc chống đông máu nào trong ít nhất vài ngày sau phẫu thuật. Nhờ bác sĩ tư vấn về việc này nếu đang sử dụng thuốc chống đông máu, vì nó làm tăng nguy cơ chảy máu hậu phẫu thuật. Tùy vào mỗi người thời gian cần ngừng uống thuốc chống đông máu sẽ khác nhau (thứ nhất là phụ thuộc vào lý do vì sao bạn phải dùng loại thuốc này). Nhờ bác sĩ hướng dẫn khi nào có thể uống thuốc trở lại bình thường. Nghỉ ngơi là một trong những yếu tố quan trọng nhất để phục hồi sau phẫu thuật cắt ống dẫn tinh. Bạn nên nghỉ phép vài ngày hoặc hạn chế làm các công việc thường nhật để vết thương lành nhanh hơn. Trừ khi công việc đòi hỏi phải hoạt động nhiều hoặc nâng nặng, bạn có thể đi làm lại khá nhanh, khoảng 2-3 ngày sau. Nếu phải làm việc nặng bạn nên hỏi bác sĩ khi nào an toàn để đi làm lại.  Cố gắng không làm việc quá nhiều trong 2-3 ngày đầu tiên sau phẫu thuật, và không ngại nhờ người khác giúp đỡ để có thời gian nghỉ ngơi phục h

In [35]:
trainer.save_model("/kaggle/working/log")

In [36]:
summarizer = pipeline("summarization", model=model, device=0,tokenizer=tokenizer)

The model 'GPT2LMHeadModel' is not supported for summarization. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'XLMProphetNetForConditionalGeneration'].


In [37]:
train["Summary"][0]

'Theo chia sẻ của TS . BS Trương Thị Ngọc Lan , Viện phó viện Y dược học dân tộc , tại Trung Quốc nhựa cây tuyết yến ( đang được rao bán trên mạng với giá cả triệu đồng / lạng ) thực chất là mủ trôm được nhập từ Lào , Việt Nam .'

In [38]:
summarizer(train["Document"][0])

Your max_length is set to 1024, but you input_length is only 598. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=299)


[{'summary_text': 'Vợ tôi thường xuyên đi làm về muộn , đó là lý do khiến tôi trở thành đầu bếp bất đắc dĩ trong nhà .Với tài nấu ăn độc nhất vô nhị , tôi đã tạo nên những thảm hoạ chưa từng có khi nỗ lực “ than hoá ” hoặc cho tất cả nguyên liệu vào nồi trong tình trạng “ nguyên thuỷ ” , không qua sơ chế .Tuy nhiên để động viên tinh thần nội trợ của chồng , vợ tôi luôn hào hứng đặt tên và chụp ảnh từng món ăn rồi khoe lên Facebook .Bộ sưu tập gồm có : Đậu Ngọc Trinh lướt bùn , ngô luộc bóng đêm , tôi thấy cua vàng trên mướp xanh , rau luộc cột đình và hàng chục cái tên kêu chan chát khác .Nhờ phát hiện chấn động về khả năng sử dụng ngôn từ của vợ mà tôi đã nghĩ ra một chiến lược kinh doanh mới , chỉ nhắm vào cái tên và không làm gì khác .Đừng tưởng rằng cái nóng mùa hè đang ảnh hưởng lên thần kinh của tôi , nếu am hiểu một chút về bán hàng trên mạng bạn sẽ thấy lời khuyên của tôi đáng giá hàng trăm triệu đồng .Nói có sách , mách có báo : Mới đây , nhiều shop online đồng loạt chào hàng 